In [11]:
import os
import pickle
import random
import math
import pandas as pd
import numpy as np
from kiwipiepy import Kiwi
import sqlite3

BASE_DIR = os.path.join('..')
DATA_DIR = os.path.join(BASE_DIR, 'data_final')
DB_DIR = os.path.join(BASE_DIR, 'database_final')

DATA_PATH = os.path.join(DATA_DIR, 'sampled_data_final.pkl')
DB_PATH = os.path.join(DB_DIR, 'search_index_final.db')

QUERIES_PATH = os.path.join(BASE_DIR, 'data', 'queries.pkl')
QRELS_PATH = os.path.join(BASE_DIR, 'data', 'qrels.pkl')

df = pd.read_pickle(DATA_PATH)

if '_id' in df.columns:
    df.set_index('_id', inplace=True)
elif 'doc_id' in df.columns:
    df.set_index('doc_id', inplace=True)
df.index = df.index.astype(str)

with open(QUERIES_PATH, 'rb') as f:
    queries_data = pickle.load(f)
with open(QRELS_PATH, 'rb') as f:
    qrels_data = pickle.load(f)

qrels_dict = {}
for item in qrels_data:
    qid, doc_id = str(item.get('query-id')), str(item.get('corpus-id'))
    if qid and doc_id:
        qrels_dict.setdefault(qid, set()).add(doc_id)

queries_map = {str(q['_id']): q['text'] for q in queries_data if '_id' in q}

kiwi = Kiwi(num_workers=-1)

def tokenize_query(text):
    try:
        return [t.form for t in kiwi.tokenize(text) if t.tag in ['NNG', 'NNP', 'VV', 'VA', 'MAG']]
    except:
        return []

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

total_docs = len(df)
avg_dl = df['doc_length'].mean()

def get_term_idf(term):
    cursor.execute('SELECT COUNT(DISTINCT doc_id) FROM inverted_index WHERE term=?', (term,))
    df_val = cursor.fetchone()[0]
    return math.log((total_docs - df_val + 0.5) / (df_val + 0.5) + 1) if df_val > 0 else 0

def calculate_scores(query_tokens, k1=5.5, b=0.99):
    scores = {}
    for term in query_tokens:
        idf = get_term_idf(term)
        if idf <= 0: continue

        cursor.execute('SELECT doc_id, tf FROM inverted_index WHERE term = ?', (term,))
        rows = cursor.fetchall()

        for doc_id, tf in rows:
            if doc_id not in df.index: continue

            doc_len = df.at[doc_id, 'doc_length']
            num = tf * (k1 + 1)
            den = tf + k1 * (1 - b + b * (doc_len / avg_dl))
            scores[doc_id] = scores.get(doc_id, 0.0) + idf * (num / den)
    return scores

valid_qids = []
for qid, docs in qrels_dict.items():
    if qid in queries_map and any(d in df.index for d in docs):
        valid_qids.append(qid)

if not valid_qids:
    print("오류: 유효한 쿼리를 찾을 수 없습니다.")
else:
    num_samples = min(10, len(valid_qids))
    test_qids = random.sample(valid_qids, num_samples)

    print(f"총 {num_samples}개의 쿼리에 대해 진단\n")

    for idx, test_qid in enumerate(test_qids):
        print("-" * 80)
        print(f"[CASE {idx + 1} / {num_samples}]")

        query_text = queries_map[test_qid]
        q_tokens = tokenize_query(query_text)
        target_docs = qrels_dict.get(test_qid, set())
        valid_targets = [d for d in target_docs if d in df.index]

        print(f"ID: {test_qid}")
        print(f"내용: {query_text}")
        print(f"토큰: {q_tokens}")
        print(f"정답 문서: {len(target_docs)}개 (샘플 내: {len(valid_targets)}개)")

        scores = calculate_scores(q_tokens, k1=5.5, b=0.99)
        ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        top_10 = ranked[:10]

        print("\n1. 상위 검색 결과 (Top 10)")
        print(f"{'순위':<4} {'점수':<8} {'문서ID':<15} {'토픽':<6} {'확률':<6} {'정답':<4} {'미리보기'}")

        ranked_ids = [d for d, s in ranked]

        for i, (doc_id, score) in enumerate(top_10):
            is_hit = "O" if doc_id in target_docs else "X"
            doc_topic = np.argmax(df.at[doc_id, 'topic_probs'])
            doc_prob = df.at[doc_id, 'topic_probs'][doc_topic]
            full_text = df.at[doc_id, 'text']
            title = df.at[doc_id, 'title'] if 'title' in df.columns else full_text[:20]
            preview = str(title).replace('\n', ' ')[:25]

            print(f"{i+1:<4} {score:<8.2f} {doc_id:<15} {doc_topic:<6} {doc_prob:<6.2f} {is_hit:<4} {preview}")

        print("\n2. 정답 문서 분석 (Ground Truth)")
        print(f"{'문서ID':<15} {'토픽':<6} {'확률':<6} {'검색순위'}")

        rel_topics = []
        for doc_id in list(valid_targets)[:5]:
            doc_topic = np.argmax(df.at[doc_id, 'topic_probs'])
            doc_prob = df.at[doc_id, 'topic_probs'][doc_topic]
            rel_topics.append(doc_topic)

            try:
                rank = ranked_ids.index(doc_id) + 1
            except ValueError:
                rank = "순위밖"

            print(f"{doc_id:<15} {doc_topic:<6} {doc_prob:<6.2f} {rank}")

        print("\n3. 진단 및 해석")
        unique_topics = set(rel_topics)
        if len(unique_topics) > 1:
            print(f"- 토픽 불일치: 정답 문서들이 서로 다른 토픽 {unique_topics}을 가짐.")
            print("  → 해석: 정답 간 공통된 토픽이 없으므로, 토픽 가중치가 0이 됨.")
        elif len(unique_topics) == 1:
            print(f"- 토픽 일치: 정답 문서들이 동일한 토픽 {unique_topics}을 가짐.")
            print("  → 해석: 그럼에도 오답 문서들과의 변별력이 없다면 토픽 정보는 무용지물")

        found_count = sum(1 for d in valid_targets if d in ranked_ids[:10])
        if found_count > 0:
            print(f"- 검색 성능: 정답 {len(valid_targets)}개 중 {found_count}개가 Top 10에 포함됨. (BM25 유효)")
        else:
            print(f"- 검색 성능: Top 10에 정답 없음. (키워드 불일치 또는 어휘 불일치 가능성)")

        print("\n")

conn.close()

총 10개의 쿼리에 대해 진단

--------------------------------------------------------------------------------
[CASE 1 / 10]
ID: query_001403
내용: 얘들아, 브롤스타즈 젬 그랩에서 없어졌다가 다시 생긴 그 돌 광산 맵 이름 뭐였지? 복귀한 거 맞나요?
토큰: ['브롤스타즈', '랩', '없', '다시', '생기', '돌', '광산', '이름', '복귀', '맞']
정답 문서: 6개 (샘플 내: 5개)

1. 상위 검색 결과 (Top 10)
순위   점수       문서ID            토픽     확률     정답   미리보기
1    85.67    브롤스타즈/맵/흔들 광산   7      0.35   O    브롤스타즈/맵/흔들 광산
2    73.61    브롤스타즈/맵/암석 광산   2      0.63   O    브롤스타즈/맵/암석 광산
3    73.29    브롤스타즈/맵/수정 오락실  2      0.56   O    브롤스타즈/맵/수정 오락실
4    66.42    브롤스타즈/맵/선인장 함정  2      0.53   O    브롤스타즈/맵/선인장 함정
5    32.14    플래티나워매몬         2      0.44   X    플래티나워매몬
6    29.92    캐피탈리즘 랩         7      0.48   X    캐피탈리즘 랩
7    28.82    가라르광산           7      0.58   X    가라르광산
8    26.72    플래시 도타/맵        2      0.36   O    플래시 도타/맵
9    26.60    멘마(나루토)         3      0.86   X    멘마(나루토)
10   24.82    카트라이더 리그/에이스 결정전/기록 9      0.42   X    카트라이더 리그/에이스 결정전/기록

2. 정답 문서 분석 (Ground Truth)
문서ID    

# [검색 시스템 심층 진단 보고서] 토픽 가중치의 한계와 BM25의 우수성 분석

## 1. 개요
본 보고서는 전체 쿼리 중 무작위로 추출한 10개 케이스(Query)를 대상으로, **BM25 검색 결과(Rank)**와 **LDA 토픽 모델링 결과(Topic Probability)**를 정성적으로 분석한 결과이다. 이를 통해 통계적 회귀분석에서 도출된 **"토픽 가중치($a_3$) 0.0"**의 원인을 실제 데이터 수준에서 규명하고, 향후 검색 모델 개선 방향을 제안한다.

## 2. 케이스별 상세 분석

분석 결과, 대다수의 케이스에서 **BM25(키워드 매칭)만으로 정답을 상위권에 정확히 노출**시키는 데 성공했다. 반면, 토픽 정보는 정답과 오답을 구별하는 변별력(Discriminative Power)을 제공하지 못하는 것으로 확인되었다.

### **패턴 A: 토픽의 변별력 부재 (Low Discriminative Power) - 다수 발생**
> **대상:** CASE 1(브롤스타즈), CASE 2(정의당), CASE 3(병자호란), CASE 4(군종신부), CASE 5(마인크래프트), CASE 6(유로파 유니버셜리스), CASE 9(판타지 여동생), CASE 10(스쿨 오브 락)

이 패턴은 정답 문서들이 특정 토픽에 일관되게 묶여 있음에도 불구하고, 토픽 가중치가 무용지물인 이유를 가장 잘 설명해준다.

* **현상:** 정답 문서들이 동일한 토픽(예: Topic 2, 7, 8 등)을 공유하고 있어, 얼핏 보면 토픽 정보가 유효해 보인다. 그러나 **상위권에 랭크된 오답 문서들 역시 동일한 토픽을 공유**하고 있다.
    * **CASE 1 (브롤스타즈):** 정답 문서들이 **Topic 2**와 **Topic 7**로 나뉘었으며, 오답인 '캐피탈리즘 랩'(Topic 7)이나 '플래티나워매몬'(Topic 2)도 같은 토픽을 공유하여 변별력이 없었다.
    * **CASE 3 (병자호란):** 정답 문서('삼전도의 굴욕', '병자호란')가 **Topic 0**으로 분류되었으나, 오답 문서('북한-만주 관계', 9위) 역시 **Topic 0** (확률 1.00)으로 분류되었다. 만약 Topic 0에 가중치를 주었다면 오답 문서의 순위도 함께 상승했을 것이다.
    * **CASE 4 (군종신부):** 정답 문서('군종 신부', '재입대')가 **Topic 0**이지만, 오답 문서('병인박해', '목사')도 모두 **Topic 0**으로 분류되었다. '종교/군사'라는 큰 범주 안에서는 정답과 오답을 구별할 수 없음을 보여준다.
    * **CASE 10 (스쿨 오브 락):** 정답('스쿨 오브 락', '잭 블랙')이 **Topic 5**이지만, 오답('YB', '오태호' 등 가수 관련 문서)들도 **Topic 5**로 분류되어, 주제만으로는 영화와 가수를 구별하지 못했다.

* **결론:** LDA 토픽은 문서를 거시적인 주제(역사, 게임, 인물 등)로 분류할 뿐, 사용자의 구체적인 질문 의도(Specific Intent)를 파악하지 못한다. 따라서 **토픽 가중치를 높이면 주제만 같고 내용은 다른 오답 문서들이 상위권에 난입하는 노이즈(Noise)가 발생**한다.

### **패턴 B: 토픽 불일치 (Topic Mismatch)**
> **대상:** CASE 7 (포켓몬스터), CASE 8 (시크교)

이 패턴은 토픽 정보를 랭킹에 반영할 경우 치명적인 부작용이 발생할 수 있음을 보여준다.

* **현상:** 하나의 질문에 대한 정답 문서들이 서로 다른 토픽으로 분류되거나, 정답 문서의 토픽 확률이 낮음에도 상위권에 랭크되는 경우이다.
    * **CASE 7 (포켓몬스터):** 정답 문서들 간 토픽이 **Topic 2, 3, 9** 등으로 분산되었다. 특정 토픽에 가중치를 부여했다면, 다른 토픽을 가진 정답 문서들은 순위 밖으로 밀려났을 것이다.
    * **CASE 8 (시크교):** 정답 문서('시크교')는 **Topic 0**이지만, 오답 문서('인도/역사') 역시 **Topic 0** (확률 1.00)으로 더 높은 토픽 확률을 가졌다. 토픽 확률에 의존했다면 오답이 정답보다 더 높은 점수를 받았을 것이다.

* **결론:** 정답 문서 간의 **주제적 일관성(Consistency)이 결여**되어 있거나, 정답과 오답이 뒤섞여 있어 토픽 정보를 신뢰할 수 없다.

---

## 3. 종합 결론: 왜 $w_{topic} = 0$ 인가?

본 진단을 통해 **토픽 가중치($a_3$)가 0으로 수렴해야 하는 논리적 근거**가 확실해졌다.

1.  **변별력의 부재:** 주제(Topic) 정보는 정답과 오답을 가르는 필터 역할을 하지 못한다. 정답과 오답은 대부분 같은 주제 카테고리 안에 공존한다.
2.  **키워드(BM25)의 압도적 우위:** 10개 케이스 중 거의 모든 사례에서 **BM25 스코어만으로 정답 문서를 1~3위 내에 정확히 위치**시켰다.
    * *예시:* CASE 5 (마인크래프트)에서 튜닝된 BM25는 '주석', '가루' 등의 세부 키워드를 정확히 매칭하여 정답 문서('GregTech 5...')를 1위로 끌어올렸다.
3.  **오히려 방해 요소:** 토픽 정보는 정답을 찾는 데 도움을 주기보다는, 주제적으로만 유사한 오답 문서를 상위권으로 끌어올리거나(패턴 A), 서로 다른 토픽을 가진 정답 문서를 누락시키는(패턴 B) **방해 요소(Distractor)**로 작용할 가능성이 크다.

---

## 4. 제언 (Future Work)

1.  **토픽 모델 제외 확정:** 현행 검색 모델에서 **LDA 토픽 확률은 랭킹 요소에서 배제**하는 것이 검색 품질 유지에 유리하다.
2.  **BM25 파라미터 최적화 유지:** 현재 튜닝된 파라미터($k_1=5.5, b=0.99$)는 10개 케이스 전반에 걸쳐 매우 우수한 성능을 보이고 있으므로, 이를 **최종 모델의 핵심 엔진으로 확정**한다.
3.  **검색 품질 향상 방향:** 향후 성능 개선을 위해서는 토픽 모델링보다는, 동의어 처리나 문맥을 파악할 수 있는 **임베딩 기반 검색(Vector Search)** 도입을 고려해야 한다. 이는 CASE 4, 7과 같이 키워드 매칭만으로는 일부 정답을 놓치는 경우를 보완해 줄 것이다.